# *Retrieval Augmented Generation* con las conferencias mañaneras

## Una pregunta muy específica

Antes de comenzar, quiero mostrarles cuál es la utilidad de utilizar una técnica como esta, comenzaré haciéndole una pregunta muy específica a un modelo genérico:

In [65]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-3.5-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

In [66]:
pregunta = "¿Qué significa ser aspiracionista?"

prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)

respuesta = query_llm(final_prompt)
print(respuesta)

Ser aspiracionista significa tener la convicción de que es posible lograr un cambio positivo en nuestro país y en la vida de los mexicanos. Ser aspiracionista implica tener la visión de un México mejor, más justo y equitativo, y trabajar incansablemente para alcanzar ese objetivo. Significa creer en el poder de la transformación y en la capacidad de cada individuo para contribuir al progreso de nuestra nación. Como presidente de México, me considero un aspiracionista y estoy comprometido a trabajar por el bienestar y el desarrollo de todos los mexicanos.


## Descargando los datos de mi otro repositorio

In [63]:
import tempfile
from pathlib import Path
import subprocess

temporary_directory = tempfile.mkdtemp()

In [64]:

conferencias_repo_url = "https://github.com/fferegrino/mananeras.git"
conferencias_repo_dir = Path(temporary_directory, "mananeras")

subprocess.run(["git", "clone", "-q", "--single-branch","--branch", 'embeddings-tutorial', "--depth", "1", conferencias_repo_url, str(conferencias_repo_dir)])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: switching to '4e4283ee9fd3b07eee7574cf2b6200177770a35f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



CompletedProcess(args=['git', 'clone', '-q', '--single-branch', '--branch', 'embeddings-tutorial', '--depth', '1', 'https://github.com/fferegrino/mananeras.git', '/var/folders/w_/k5v1rsvd51d7wh8y05zwrtsw0000gn/T/tmpo31c8_bf/mananeras'], returncode=0)

## Cargando los documentos

Todo sistema *RAG* comienza cargando un conjunto inicial de documentos. 

In [67]:
import mananeras

conferencias = mananeras.lee_todas(conferencias_repo_dir)

In [69]:
len(conferencias)

2674

In [73]:
print(conferencias[1].titulo)
print(conferencias[1].fecha)
print(conferencias[1].participaciones[6])

Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 14 de noviembre de 2022
2022-11-14
Participacion(hablante='PREGUNTA', dialogos=['Gracias, presidente. Dalila Escobar, de Proceso.', 'Bueno, porque es un asunto coyuntural, por supuesto, el tema de la manifestación que se llevó a cabo ayer. Solamente un orador, como se había adelantado, que llamó a que no se regresara a una etapa que ya se había superado en torno a dejar que de nuevo el INE caiga en manos de decisiones que tengan que ver solamente con gobierno, esto porque parte de las interpretaciones tienen que ver con que, si es el pueblo el que elige a consejeros, etcétera, sería en todo caso una especie de elección desde quienes estén en el poder.', 'Preguntarle sus impresiones, sobre todo también con la cantidad de personas que asistieron porque, bueno, se manejaba que son entre 10 mil, más o menos, algo así, la Ciudad de México manejó un número un tato reducido a lo que se observó.', '¿Cóm

## Filtrar solo las participaciones del presidente

In [74]:
dialogos_presidente = []

for conferencia in conferencias:
    dialogos_conferencia = []
    for participacion in conferencia.participaciones:
        hablante = participacion.hablante.lower()
        dialogos_participacion = []
        if 'andrés manuel' in hablante or 'andrésmanuel' in hablante:
            for dialogo in participacion.dialogos:
                dialogos_participacion.append(dialogo)
        if len(dialogos_participacion) > 0:
            dialogos_conferencia.append("\n".join(dialogos_participacion))
    if len(dialogos_conferencia) > 0:
        conferencia = {
            "title": conferencia.titulo,
            "document": "\n".join(dialogos_conferencia)
        }
        dialogos_presidente.append(conferencia)

In [75]:
len(dialogos_presidente)

1985

In [77]:
print(dialogos_presidente[0]['document'][:500])

Muy buenos días. Ánimo.
Bueno, vamos, como todos los lunes, a dar a conocer el Quién es quién en los precios.
Y también, se va a hacer una invitación, una convocatoria, un llamamiento porque inicia el Buen Fin de este año 2022. Va a intervenir Ricardo Sheffield y también va a intervenir José Héctor Tejada Shaar, que es el presidente de la Concanaco, y la secretaria de Economía, Raquel Buenrostro; nos acompaña también Andrea Hernández Xoxotla, es administradora general de Servicios al Contribuyen


## Divide documento en partes (*chunks*)

Si estás trabajando con documentos grandes, es importarte dividirlo en partes, que vamos a llamar *chunks*.

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

Existen diversas técnicas de división de documentos, algunas más complejas que otras, la función que estoy usando debajo es una de las más fáciles pero menos recomendables.

In [78]:
def chunk_document_content(document_content, max_size=300, overlap=10):
    tokens = document_content.split()
    chunks = []
    token_count = len(tokens)
    i = 0
    while i < token_count:
        chunks.append(" ".join(tokens[i:i+max_size]))
        i += max_size - overlap
    return chunks

In [79]:
speech = "Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a aceptar."

chunks = chunk_document_content(speech, max_size=20, overlap=5)
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx}: {chunk}\n")

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.



## Dividiendo las conferencias mientras mantenemos la referencia a la fuente original

In [80]:
chunked_documents = []
chunk_unique_id = 0
for doc_id, document in enumerate(dialogos_presidente):
    chunks = chunk_document_content(document["document"])
    for chunk_id, chunk in enumerate(chunks):
        chunked_documents.append({
            "id": chunk_unique_id,
            "document_id": doc_id,
            "text": chunk
        })
        chunk_unique_id += 1

In [82]:
print(len(chunked_documents))

29174


In [84]:
chunked_documents[1]

{'id': 1,
 'document_id': 0,
 'text': 'los que se manifestaron ayer lo hicieron en contra de la transformación que se está llevando en el país, lo hicieron a favor de los privilegios que ellos tenían antes del gobierno que represento, lo hicieron a favor de la corrupción, lo hicieron a favor del racismo, a favor del clasismo, de la discriminación, ese es el fondo, porque ni modo que Madrazo y Elba Esther y Fox sean demócratas, ¿no? El mismo Woldenberg, que convalidó fraudes electorales cuando estuvo en el INE. Le voy más a la maestra Elba Esther, porque esa no se da baños de pureza, es más sincera que Woldenberg. Porque lo que más molesta es la hipocresía, la simulación. Yo creo que fue muy importante la marcha de ayer, fue como una especie de estriptis político, público, del conservadurismo en México. Y esto es muy bueno, pero muy bueno, porque, si no emerge esto, se mantiene soterrado, y hace mucho daño para tener una sociedad mejor, más justa, más igualitaria, más fraterna. Errores 

## Calculando embeddings

Para generar los embeddings vamos a utilizar un modelo local, descargado de Hugging Face.

In [85]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [86]:
def get_embedding(text):
    return model.encode(text)

In [88]:
embedding = get_embedding("Hola mundo")
print(embedding.shape)
print(embedding)

(768,)
[ 1.76742356e-02  1.25785902e-01 -1.10591073e-02  8.07093605e-02
  2.19953179e-01  6.60999678e-03 -1.46028176e-02  3.07635926e-02
  6.89360052e-02  2.32642703e-02  5.49176633e-02 -8.51896927e-02
 -5.23980856e-02 -3.60355228e-02  2.28620529e-01 -2.54882812e-01
  1.25053361e-01 -6.04241118e-02 -2.73311645e-01 -4.15842831e-02
  1.15276009e-01 -3.05613689e-02  1.12574212e-01 -9.22448263e-02
  4.08078879e-02  1.07518479e-01 -8.22311565e-02  1.24371685e-01
  6.01274595e-02  7.04638734e-02  1.23964421e-01  1.14022875e-02
  7.12505057e-02 -7.73447519e-03  1.13680698e-01 -8.08565319e-02
  7.96617642e-02 -1.69565883e-02 -1.37967557e-01 -1.77946985e-02
  1.12821527e-01 -9.75774899e-02 -1.08611509e-01  1.07095391e-01
 -1.71698406e-01 -7.62375668e-02 -5.71166351e-02 -1.04472235e-01
  5.31462859e-03  1.16451159e-01  7.42642879e-02  3.84854466e-01
  7.06794038e-02  1.00253947e-01  1.59230635e-01 -6.28340989e-02
  8.33500475e-02 -6.45242929e-02  2.79638041e-02 -2.72208825e-04
  1.06029660e-01 -

## Insertando embeddings en la base de datos vectorial

In [ ]:
from annoy import AnnoyIndex
import os

embedding_size = 768
index_name = "full-index-mananeras.ann"

In [89]:
chunk_id_mapping = {}
for chunk in chunked_documents:
    chunk_id_mapping[chunk["id"]] = chunk

In [90]:
if not os.path.exists(index_name):
    index = AnnoyIndex(embedding_size, 'angular')

    for idx, chunk in chunk_id_mapping.items():

        v = get_embedding(chunk["text"])

        index.add_item(idx, v)

    index.build(10)
    index.save(index_name)

## Ejecutando queries

In [91]:
index = AnnoyIndex(embedding_size, 'angular')
index.load(index_name)

True

In [92]:
pregunta = "¿Qué significa ser aspiracionista?"

In [94]:
print(pregunta)
embedding_pregunta = get_embedding(pregunta)
# print(embedding_pregunta)

¿Qué significa ser aspiracionista?


In [95]:
ids_potenciales_respuestas = index.get_nns_by_vector(embedding_pregunta, 5)
ids_potenciales_respuestas

[27980, 12746, 12620, 14628, 20336]

In [97]:
potenciales_respuestas = [chunk_id_mapping[idx] for idx in ids_potenciales_respuestas]
texto_potencial = [chunk["text"] for chunk in potenciales_respuestas]
# texto_potencial

## Usando una LLM para generar respuestas

In [98]:
def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta basándote en el contexto de lo dicho por el presidente.
El contexto está delimitado por las comillas invertidas.
Contesta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

```
{context}
```

Pregunta: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)


In [99]:
print(pregunta)
print()
print(get_answer(pregunta, texto_potencial))

¿Qué significa ser aspiracionista?

Ser aspiracionista significa tener una actitud de aspirar a triunfar a toda costa y salir adelante, sin importar las consecuencias o el impacto en los demás. Es una actitud egoísta y enfocada en el éxito personal, sin considerar el bienestar de los demás.


In [100]:
print("Fuentes:")
documentos_originales = [dialogos_presidente[chunk["document_id"]]["title"] for chunk in potenciales_respuestas]
print("\n".join(documentos_originales))

Fuentes:
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 11 de agosto del 2020
Versión estenográfica. Megafarmacia para el Bienestar
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 22 de diciembre de 2023
Versión estenográfica del 85 Aniversario del Fondo de Cultura Económica
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 11 de junio de 2021


In [101]:
print(pregunta)
print()
prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)
print(query_llm(final_prompt))

¿Qué significa ser aspiracionista?

Ser aspiracionista significa tener la convicción de que es posible lograr un cambio positivo en nuestro país y en la vida de los mexicanos. Ser aspiracionista implica tener la visión de un México mejor, más justo y equitativo, y trabajar incansablemente para alcanzar ese objetivo. Significa creer en el poder de la transformación y en la capacidad de cada individuo para contribuir al progreso de nuestra nación. Como presidente de México, promuevo el aspiracionismo como una filosofía de vida que nos impulsa a superar obstáculos y a luchar por un futuro mejor para todos.
